<a href="https://colab.research.google.com/github/jmuddappa/DeepClassificationBot/blob/master/roberta_cdqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!git clone https://github.com/pytorch/fairseq

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [36]:
%cd fairseq
!pip install --editable .

/content/fairseq
Obtaining file:///content/fairseq
  Found existing installation: fairseq 0.9.0
    Can't uninstall 'fairseq'. No files were found to uninstall.
  Running setup.py develop for fairseq


In [0]:
!wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
!tar -xvf /content/fairseq/roberta.large.tar.gz
!bash examples/roberta/commonsense_qa/download_cqa_data.sh


--2020-02-14 19:57:59--  https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655283069 (625M) [application/gzip]
Saving to: ‘roberta.large.tar.gz.1’

roberta.large.tar.g  62%[===========>        ] 393.70M  12.2MB/s    eta 21s    

In [0]:
%env MAX_UPDATES=3000
%env WARMUP_UPDATES=150
%env LR=1e-05
%env MAX_SENTENCES=16
%env SEED=1
%env ROBERTA_PATH=/roberta.large/model.pt
%env DATA_DIR=/content/fairseq/data/CommonsenseQA

%env FAIRSEQ_PATH=/content/fairseq
%env FAIRSEQ_USER_DIR=/content/fairseq/examples/roberta/commonsense_qa

!CUDA_VISIBLE_DEVICES=0 fairseq-train --fp16 --ddp-backend=no_c10d $DATA_DIR --user-dir $FAIRSEQ_USER_DIR \
    --restore-file $ROBERTA_PATH \
    --reset-optimizer --reset-dataloader --reset-meters \
    --no-epoch-checkpoints --no-last-checkpoints --no-save-optimizer-state \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --task commonsense_qa --init-token 0 --bpe gpt2 \
    --arch roberta_large --max-positions 512 \
    --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
    --criterion sentence_ranking --num-classes 5 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 1e-06 --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr $LR \
    --warmup-updates $WARMUP_UPDATES --total-num-update $MAX_UPDATES \
    --max-sentences $MAX_SENTENCES \
    --max-update $MAX_UPDATES \
    --log-format simple --log-interval 25 \
    --seed $SEED